# Ajout de Variables Supplémentaires 📊

Ajout de modalités supplémentaires à l'ensemble de données de séries temporelles existant.

Nous utilisons le package `pandarallel` pour accélérer les calculs de l'angle du soleil.  

## Initialisation

### Importation des packages pythons

In [ ]:
# std
import os
import inspect
import json
# external
import numpy as np
import pandas as pd
from pandarallel import pandarallel
import missingno as msno
# local
from utils import get_astro_sunAlt

In [ ]:
# Active la recharge automatique de modules dans un Jupyter Notebook
%load_ext autoreload
# %autoreload 0 : Désactive la recharge automatique.
# %autoreload 1 : Recharge automatiquement les modules importés avec %aimport uniquement.
# %autoreload 2 : Recharge tous les modules (sauf ceux exclus par %aimport), à chaque fois que du code est exécuté.
%autoreload 2

### Configuration des packages

In [ ]:
pandarallel.initialize(progress_bar=True)

### Paramètres

In [ ]:
dataversion = "data-2310-Yewan-MissingValues-20231106"  # Data version.
force_sun = False # If we should force to run sun position (could be long)
n_samples = None # For debugging use n_sample >= n_cpu or None

In [ ]:
data_directory = os.getcwd() + os.sep  + "data" + os.sep + dataversion  # Directory where we put the data.
path_data_1m = data_directory + os.sep + "tenerife_2020_1m.csv" # Path of the 1 minute sampled data.
path_data_sun1m = data_directory + os.sep + "tenerife_2020_sun1m.csv" # Path of the 1 minute sampled data with sun altitude.
path_data_ext =  data_directory + os.sep + 'tenerife2020_extended.csv'  # Path to extended dataset.
location = "tenerife" # Localisation of the ISM.  
dic_location = {
    'tenerife': {
        'lat': 28.3005372, 
        'lon': -16.513448, 
        'height': 731, 
        'timezone': 'Atlantic/Canary'
    }
}

In [ ]:
if n_samples:
    # Change output file in order to keep preprocessed data
    path_data_sun1m = path_data_sun1m.replace("_sun1m.csv", f"_sun1m{n_samples}.csv")
    path_data_ext = path_data_ext.replace("_extended.csv", f"_extended{n_samples}.csv")

### Fonctions pour faciliter le traitement

In [ ]:
print(inspect.getsource(get_astro_sunAlt))

## Lecture des données échantilloner à 1 minute

In [ ]:
df = pd.read_csv(path_data_1m, index_col="time")
df.index = pd.to_datetime(df.index)

### Si besoin reduction du nombre de données pour aller plus vite

In [ ]:
# Reducing the number of row if needed (for dev)
if n_samples:
    df = df.sample(n_samples).copy()
else:
    n_samples = len(df)

### Vérification des données lus

In [ ]:
df.info()

In [ ]:
df.sample(min(5, n_samples)).T

## Création de nouvelle variables

Ajout de caractéristiques supplémentaires à l'ensemble de données de séries temporelles existant, afin d'améliorer la performance de prédiction du modèle de prédiction.
- Ajouter l'altitude du soleil
- Ajouter la temporalité : mois, saison, heure de la journée

L'ajout du calcul de la position au soleil peux être long. Si c'est le cas le fichier pré-processer peux être utilisé.  

### Ajouter l'altitude du soleil

L'altitude du soleil : Indicateur crucial pour les variations thermiques, et détermine le jour et la nuit.

Calculer l'altitude du soleil est computationnellement intensif, le calcul parallèle est envisagé pour accélérer le calcul.


In [ ]:
def get_sunalt(x):
    ''' Wrapper for use with pandarallel '''
    return get_astro_sunAlt(dic_location[location], x)

# force_sun = True

if os.path.isfile(path_data_sun1m) and not force_sun:
    df = pd.read_csv(path_data_sun1m, index_col="time")
    df.index = pd.to_datetime(df.index)
else:
    df['time'] = df.index
    df['sun_alt'] = df["time"].parallel_apply(get_sunalt)
    df.drop("time", axis=1, inplace=True)
    df.to_csv(path_data_sun1m, index=True)

### Temporalité

In [ ]:
df['hour_of_day'] = df.index.hour
df['month'] = df.index.month
df['season'] = (df['month'] % 12 + 3) // 3  # 1: Winter, 2: Spring, 3: Summer, 4: Fall
season_decode = {
    "1": "Hiver",
    "2": "Printemp",
    "3": "Été",
    "4": "Automne"
}

In [ ]:
with open('season.json', 'w') as f:
    json.dump(season_decode, f)

## Vérification

### Statistique descriptives

In [ ]:
df.info()

In [ ]:
df.describe(include="all").T

### Vizualisation

In [ ]:
ax = df.month.plot(style='.', title="month")

In [ ]:
ax = df.hour_of_day.plot(style='.', title="hour")

In [ ]:
ax = df.season.plot(style='.', title="season")
print(season_decode)

In [ ]:
ax = df.sun_alt.plot(style='.', title="orientation du soleil")

In [ ]:
off_d = 3
off = off_d*24*60
ax = df.iloc[off:off+60*24*2].sun_alt.plot(style='-.',title="orientation du soleil")

In [ ]:
cols = sorted(df.columns.tolist())
ax = msno.matrix(df[cols])

## Échantillon de test

In [ ]:
dfo = df.copy()

In [ ]:
mask_d = ~dfo.day_r0.isnull()
n_d = sum(mask_d)
print(f"Le nombre de valeurs ou r0 de jour est non null est {n_d}")
mask_n = ~dfo.night_r0.isnull()
n_n = sum(mask_n)
print(f"Le nombre de valeur ou le r0 de nuit est non null est {n_n}")
mask = (mask_d & mask_n)
assert sum(mask) == 0
ns_d = 1000
ns_n = 1000
f_d = ns_d/n_d
f_n = ns_n/n_n
print(f"Le ratio de l'échantillon pour la nuit sont {f_n*100:.04f}% et pour le jour {f_d*100:.04f}%")
dfo_test_d = dfo[mask_d].sample(n=ns_d, random_state=42).copy()
dfo_test_n = dfo[mask_n].sample(n=ns_n, random_state=42).copy()
dfo.loc[dfo_test_d.index, "day_r0"] = np.nan
dfo.loc[dfo_test_d.index, "night_r0"] = np.nan

## Sauvegarde des données

In [ ]:
path_data_ext_testday = path_data_ext.replace("extended.csv", "extended_testday.csv")
dfo_test_d.to_csv(path_data_ext_testday, index=True)

In [ ]:
path_data_ext_testnight = path_data_ext.replace("extended.csv", "extended_testnight.csv")
dfo_test_n.to_csv(path_data_ext_testnight, index=True)

In [ ]:
dfo.to_csv(path_data_ext, index=True)